# Final Retrieval System

This is a (slightly) improved retrieval system based on the BM25 retrieval.
It implements a custom index, query extension and stemming, sequential dependence and a simple pipeline.

### Step 1: Import Libraries

Imports TIRA, PyTerrier and Spacy and ensures PyTerrier is loaded.

In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

import spacy

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Setup for Stopwords

Loads English stopwords from Spacy. Saves them in the file "stopwords.txt" and changes the PyTerrier stopwords file to that. 

In [2]:
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)


file_path = "stopwords.txt"
with open(file_path, 'w+') as file:
    for element in spacy_stopwords:
        file.write(element+ "\n")

pt.set_property('stopwords.filename','./stopwords.txt')

### Step 3: Import Dataset

Imports the ir_dataset from TIRA and prints the first two queries to see if it worked.

In [3]:
data = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

See the first two queries:
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.47MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/
  qid                                     query
0   1  retrieval system improving effectiveness
1   2  machine learning language identification


Function to lemmatize text

In [9]:
# Function to lemmatize text
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.text.lower() not in spacy_stopwords])



### Step 4: Build an Index with lemmatization

Builds a custom iterable index. 
Stems words and removes stopwords previously defined.
Block indexing is enabled.

In [10]:
class LemmaIndexer(pt.IterDictIndexer):
    def process(self, doc):
        doc['text'] = lemmatize_text(doc['text'])
        return doc

indexer = LemmaIndexer("/tmp/index", overwrite=True, blocks=True, meta={'docno': 100, 'text': 20480})
index_ref = indexer.index(data.get_corpus_iter())
print('Done. Index is created')

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  65%|██████▌   | 82794/126958 [00:16<00:08, 5239.63it/s]

17:47:41.834 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1980.sigirconf_conference-80.24) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:23<00:00, 5404.06it/s] 


17:47:52.052 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents
Done. Index is created


### Step 5: Setup for Retrieval-Pipeline

First definitions of index and retrieval models used.
"sdm" enables use of Sequential Dependance Model.

In [5]:
index = pt.IndexFactory.of(index_ref)
bm25 = pt.BatchRetrieve(index, wmodel="BM25", k1=1.5, b=0.75, verbose=True)
pl2 = pt.BatchRetrieve(index, wmodel="PL2", c=7.0, verbose=True)
sdm = pt.rewrite.SequentialDependence()

Secondly, definition of the query expansion and the actual pipeline.

In [6]:
#Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index) 
rm3 = pt.rewrite.RM3(index)

#Pipeline
pipe = (bm25 % 100) >> rm3 >> sdm >> pl2

### Step 6: Create the Run

Creates the run on the queries from the dataset.

In [7]:
print('Create run')

run = pipe(topics)

print('Done, run was created')

Create run


BR(BM25): 100%|██████████| 68/68 [00:12<00:00,  5.57q/s]


17:38:24.488 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 6 possibilities
17:38:24.670 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 5 possibilities


BR(PL2): 100%|██████████| 68/68 [00:13<00:00,  4.93q/s]


Done, run was created


### Step 7: Save the Runfile

In [8]:
persist_and_normalize_run(run, 'retrieval_system', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
